#Task 1: Tokenization



In [91]:
# Download gutenberg dataset
# This is the dataset on which all your models will be trained
# https://drive.google.com/file/d/0B2Mzhc7popBga2RkcWZNcjlRTGM/edit

In [54]:
import sys, re
import string
import glob
import math

# ------------------------------------ Tokenize sentence -------------------------------------------------------
def replace_char(p,r,text):
    t = ""
    if r == 1:
        t =" "
    if r == 2:
        t ==". "
    for x in p:
        text = text.replace(x,t)
    return text

def replace_digit(text):
    for x in text:
        if x.isdigit():
            text = text.replace(x,"")
    return text

def handling_newlines(text):
    text = text.replace("\n\n" , ". ")
    text = text.replace(".\n\n" , ". ")
    text = text.replace(".." , ".")
    return text

def tokenize(text,sent,tok):
    # Given text, tokenise into sentences and words. Take into consideration various delimiters.
    text = text.lower()
    
    px = ["[","]","'s","(",")","\"","%","$","*","&","^","=","+","`","'"] 
    text = replace_char(px,0,text)
    text = replace_digit(text)
    text = handling_newlines(text)    
    p = [":--","-","\n","  ","_",",",";",":","/","{","}"]
    text = replace_char(p,1,text)
    p1 = ["!","?"]
    text = replace_char(p1,2,text)
    
    sentences = []
    s = "<SOS> "
    for word in text:
        if word.endswith('.'):
            word = word[:-1]
            s = s + word+" <EOS>"
         
            if s != "<SOS>  <EOS>":
                s = s.replace("  "," ")
                sentences.append(s.strip())
            s = "<SOS> "
        
        else:
            s = s + word
            
    for sent1 in sentences:
        if sent1 == " " or sent1 == "":
            remove(sent1)
            
    sent.extend(sentences)        
    tokens = text.split()
    lower_tokens = []
    for x in tokens:
        if x == '' or len(x)==0:
            print("hi")
            continue
        lower_tokens.append(x.lower())
    punct = string.punctuation
    table = str.maketrans('','',punct)
    stripped = [w.translate(table) for w in tokens]
    tok.extend(stripped)
    return tok,sent

def get_tokens():
    sentences = []
    tokens = []
    path = '/home/rahil/Desktop/Gutenberg/txt/*.txt'
    files = glob.glob(path)
    i = 0
    for name in files:
        file = open(name, 'rt')
        text = file.read()
        tokens,sentences = tokenize(text,sentences,tokens)
        i+=1
    return tokens,sentences


Language Model and Smoothing


In [55]:
# ---------------------------------- Build your language model ---------------------------------------------
def language_model(tokens,sentences):
    global unigramDict
    global bigramDict    
    trigramLangModel(tokens,sentences) 

In [56]:
def unigramLangModel(words):
    global vocabSize   
    for word in words:
        if len(word) == 0:
            continue
        if word != startOfSentence and word != endOfSentence:
            unigramDict[word] = unigramDict.get(word,0) + 1
            vocabSize += 1 
        
def bigramLangModel(a,sentences):
    unigramLangModel(a)
    unique_bigrams = set()
    for sentence in sentences:
        sentence = sentence.split()
        
        previous_word = None
        for word in sentence:           
            if previous_word != None:
                bigramDict[(previous_word,word)] = bigramDict.get((previous_word,word),0) + 1

            if previous_word != startOfSentence and previous_word != endOfSentence:
                unique_bigrams.add((previous_word,word))
            previous_word = word
    
def trigramLangModel(a,sentences):
    bigramLangModel(a,sentences)
    unique_trigrams = set()
    for sentence in sentences:
        sentence = sentence.split()
        prev1_word = None
        prev2_word = None
        for word in sentence:
            if prev1_word != None and prev2_word != None:
                trigramDict[(prev1_word,prev2_word,word)] = trigramDict.get((prev1_word,prev2_word,word),0) + 1

                if prev1_word != startOfSentence and prev1_word != endOfSentence:
                    unique_trigrams.add((prev1_word,prev2_word,word))
            prev1_word = prev2_word
            prev2_word = word
   

In [57]:
#Complete Ngram function

In [58]:
# Given (N-1) gram, and the value 'N', print the possibilities that complete the n-gram
# and plot them in decresing order of frequency

def cmp(l1,l2):
    if len(l1) != len(l2):
        return -1
    for i in range(0,len(l1)):
        if l1[i] != l2[i]:
            return -1
        
    return 0

def completeNgram(ngram, n):
    temp1 = {}
    if n==2:
        temp = ngram[-(n-1):]
        for k,v in bigramDict.items():
                if cmp(k[:-1],temp) == 0:
                    temp1[k[-1]] = v
        
    elif n>=3:
        temp = ngram[-2:]
        for k,v in trigramDict.items():
                if cmp(k[-3:-1],temp) == 0:
                    temp1[k] = v
    else:
        pass
    
    temp2 = sorted(temp1.items(),key = itemgetter(1),reverse = True)
    print("\n--------------- completeNgram ----------------\n")
    print("When " + str(ngram) + " was given\n")
    
    for x in temp2:
        print(str(x))
    

In [59]:
# ------------------------------------- Perform Stupid Backoff smoothing ------------------------------------------
def stupidBackOff(word,sentence):
    finalResult = 0.0
    
    for i in range(len(sentence)):
        if sentence[i]!='<SOS>' and sentence[i]!='<EOS>' and sentence[i-1]!='<SOS>': 
            bi_key = (sentence[i - 1],sentence[i])
            count_bigram = bigramDict.get(bi_key,0)
            count_unigram = unigramDict.get(sentence[i - 1],0)
            
            if count_bigram > 0:
                finalResult += math.log(count_bigram)
                finalResult -= math.log(count_unigram)
            
            else:
                count_unigram = unigramDict[sentence[i]]
                finalResult += math.log(count_unigram + 1)
                finalResult -= math.log(vocabSize * 2)
                finalResult += math.log(0.4)
    
    return finalResult


In [60]:
# ------------------------------------- Perform Laplace smoothing ------------------------------------------
def laplaceSmoothing(num,denum,n):
    
    num += 1    
    if n == 2:
        denum += len(unigramDict) + 1
    elif n == 3:
        denum += len(bigramDict) + 1
    return num,denum

In [61]:
# ---------------------------------- Perform Good Turing smoothing -----------------------------------------
def goodTuring(dict1):  
    nrs = {}
    for k, v in dict1.items():                                 # reverse indexing
        if v not in nrs:
            nrs[v] = []
        nrs[v].append(k)
        
    nr_counts = {k : len(v) for k, v in nrs.items()}
    nr_probs = {k : (k*v)/float(N) for k, v in nr_counts.items()}
    sorted_nrs = sorted(nr_counts.items())
    sorted_probs = sorted(nr_probs.items())
    MAX = sorted_nrs[0][1]
            
    for r, nr in nrs.items():
        if (r+1) in nrs:
            #print(nrs[r+1])
            new_nr = float((r+1) * len(nrs[r+1])) / float(N) 
        else:
            new_nr = MAX*r**-2 / float(N)
        
    return new_nr

In [62]:
# --------------------------------------- Deleted Interpolation --------------------------------------------
import numpy as np
def deleted_interpolation(unigram_c,bigram_c,trigram_c):
    lambda1 = 0
    lambda2 = 0
    lambda3 = 0
    for a, b, c in trigram_c.keys():
        if a != '<SOS>':
            v = trigram_c[list(a, b, c)]
            if v > 0:
                try:
                    c1 = float(v-1)/(bigram_c[list(a, b)]-1)
                except ZeroDivisionError:
                    c1 = 0
                try:
                    c2 = float(bigram_c[list(a, b)]-1)/(unigram_c[a]-1)
                except ZeroDivisionError:
                    c2 = 0
                try:
                    c3 = float(unigram_c[a]-1)/(sum(unigram_c.values())-1)
                except ZeroDivisionError:
                    c3 = 0            
                clist = [c1, c2, c3]
                m = np.argmax(clist)
                if m == 0:
                    lambda3 += v
                if m == 1:
                    lambda2 += v
                if m == 2:
                    lambda1 += v

    weights = [lambda1, lambda2, lambda3]
    weights = [float(a) / sum(weights) for a in weights]
    
    print(weights)
    return weights

def cal_probs():
    unigram_total = sum(unigramDict.values())

    for k in unigramDict.items():
        unigram_p[k] = math.log(unigramDict[k], 2) - math.log(vocabSize, 2) if k != "<SOS>" and k != '' and len(k)>0 else 0
        
    for k in bigramDict.items():
        bigram_p[k] = math.log(bigramDict[k], 2) - math.log(unigramDict[k[0]], 2) if k[0] != "<SOS>" else 0
    
    for k in trigramDict.items():
        trigram_p[k] = math.log(trigramDict[k], 2) - math.log(bigramDict[k[:-1]], 2) if k[0] != "<SOS>" else 0
    
    weights = deleted_interpolation(unigramDict, bigramDict, trigramDict)
    trigram_d = math.log(weights[0] * (2**trigram_p) + weights[1] * (2**bigram_p) + weights[2] * (2**unigram_p), 2)
    return trigram_d

#Task 2: Unigrams and Spelling Correction

In [63]:
# ---------------------------------- Top 10 and Bottom 10 frequencies ----------------------------------
def ngramFreqPlot(ngrams):
    
    temp =  {}
    for key,value in ngrams.items():
        temp_str = str(key)
        temp[temp_str] = value
    
    # Calculate unigram frequencies and plot them in the descending order of frequency
    temp = OrderedDict(sorted(temp.items(),key = itemgetter(1),reverse = True))       
    word = list(temp.keys())
    freq = list(temp.values())
    plot_ngram_freqs(word,freq)
    
    w = []
    
    print("\n----------------- Top 10 ---------------\n")
    w = sorted(temp.items(),key = itemgetter(1),reverse = True)
    for i in w[:10]:
         print(str(i[0]) + " --> " + str(temp[i[0]]))
    
    print("\n----------------- Bottom 10 ---------------\n")
    for i in w[len(temp)-10:]:
        print(str(i[0]) + " --> " + str(temp[i[0]]))
    

In [64]:
# --------------------------------- Plot ngrams vs freq plot ----------------------------------------------
def plot_ngram_freqs(word,freq):
    
    # Create and style traces
    trace0 = go.Scatter(x = word,y = freq,name = 'Plot Ngrams_freq',line = dict(color = ('rgb(205, 12, 24)'),width = 2))   
    data = [trace0]

    # Edit the layout
    layout = dict(title = 'Plot of ngrams vs frequency',xaxis = dict(title = 'word'),yaxis = dict(title = 'frequency',type='log',autorange=False))

    fig = dict(data=data, layout=layout)
    py.iplot(fig, filename='styled-line')


In [65]:
# --------------------------------------- Spelling checker ------------------------------------------
# Given a word check if the spelling is correct using your Language model
#     probabilistic_spellcorrection("across")

def probabilistic_spellcorrection(word):
#     print(a)
    char_unigram = {}
    for cu in word:
        for l in cu:
            char_unigram[l] = char_unigram.get(l,0) + unigramDict[cu]
        
    char_bigram = {}
    for cb in word:
        for index in range(0,len(cb)-1):
            char_bigram[(cb[index],cb[index+1])] = char_bigram.get((cb[index],cb[index+1]),0) + freq_dict_unigram[cb]
            
    print(char_unigram)
    print("\n\n")
    print(char_bigram)

def spellCheck(word, n_grams):    
    word = word.lower()
    return (set(edit_distance(word)) & set(unigramDict.keys()))
    
def edit_distance(word):
    alphabet = set('abcdefghijklmnopqrstuvwxyz')
    
    splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [a + b[1:] for a, b in splits if b]
    transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
    replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
    inserts    = [a + c + b for a, b in splits for c in alphabet]
    return set(deletes + transposes + replaces + inserts)


#Task 3 : Grammaticality Test

In [66]:
def unigramProb(word,smoothing,sm):
    num = unigramDict.get(word,0)
    denum = vocabSize
    
    if smoothing:
        if sm == 1:
            num,denum = laplaceSmoothing(num,denum,1)
        elif sm == 2:
            return goodTuring(unigramDict)
        elif sm == 3:
            return stupidBackOff(word,sent)
    if num == 0 or denum == 0:
        return 0.0
    else:
        return float(num) / float(denum)

def unigramSentCheck(sent):
    unigrams = []
    temp = []
    unigrams,temp = tokenize(sent,unigrams,temp)
    sum = 0.0

    for uni in unigrams:
        pro = unigramProb(uni,True,2)
        if pro!=0:
            sum += pro
    return sum

In [67]:
# new_nrs = goodTuring(nr_counts)
# sorted_newnrs = sorted(new_nrs.items())
# print(sorted_newnrs)

def bigramProb(prev_word,word,smoothing,sm):
    num = bigramDict.get((prev_word,word),0)
    denum = unigramDict.get(prev_word,0)
    
    if smoothing:
        if sm == 1:
            num,denum = laplaceSmoothing(num,denum,2)
        elif sm == 2:
            return goodTuring(unigramDict)
        elif sm == 3:
            return stupidBackOff(word,sent)
    if num == 0 or denum == 0:
        return 0.0
    else:
        return float(num)/float(denum)

def bigramSentCheck(sent):
    sum = 0.0
    previous_word = None
    sent = sent.split()
    for word in sent:
        if previous_word != None:
            bigram_prob = bigramProb(previous_word,word,True,2)
            if bigram_prob!=0:
                sum += bigram_prob
        previous_word = word
    return sum       

In [68]:
def trigramProb(prev1_word,prev2_word,word,smoothing,sm):
    num = trigramDict.get((prev1_word,prev2_word,word),0)
    denum = bigramDict.get((prev1_word,prev2_word),0)
    
    if smoothing:
        if sm == 1:
            num,denum = laplaceSmoothing(num,denum,3)
        elif sm == 2:
            return goodTuring(bigramDict)
        elif sm == 3:
            return stupidBackOff(word,sent)
        
    if num == 0 or denum == 0:
        return 0.0
    else:
        return float(num)/float(denum)
    
def trigramSentCheck(sent):
    sum = 0.0
    unique_trigrams = set()
    sent = sent.split()
    prev1_word = None
    prev2_word = None
    for word in sent:
        if prev1_word != None and prev2_word != None:
            trigram_prob = trigramProb(prev1_word,prev2_word,word,True,2)
            sum += trigram_prob
        
        prev1_word = prev2_word
        prev2_word = word
    return sum 

Main Function

In [69]:
# ------------------------------------ MAIN FUNCTION -------------------------------------------------------
from collections import OrderedDict
from operator import itemgetter
import plotly.plotly as py
import plotly.graph_objs as go

unigramDict = {}
bigramDict = {}
trigramDict = {}

vocabSize = 0
temp_dict = {}

startOfSentence = "<SOS>"
endOfSentence = "<EOS>"

tokens,sentences = get_tokens()                                                       # Tokenizing
language_model(tokens,sentences)                                                      # Building Language Model
N = sum(unigramDict.values())




In [77]:
# ------------------------------------ Log plots of frequencies and ngrams ---------------------------------------

ngramFreqPlot(unigramDict)                                    # Print top 10 and bottom 10 unigrams
ngramFreqPlot(bigramDict)                                     # Print top 10 and bottom 10 bigrams
ngramFreqPlot(trigramDict)                                    # Print top 10 and bottom 10 trigrams

PlotlyRequestError: No message

In [75]:
# ------------------------------------ Spell Checking and Correction ------------------------------------------------
#probabilistic_spellcorrection("dairy")
possibilities = spellCheck("dairy",unigramDict)              # SpellChecking

print("Suggestions for the misspelled word : ")
print(possibilities)                                                     # Possible words
print("\n")

Suggestions for the misspelled word : 
{'fairy', 'airy', 'diary', 'daily', 'hairy'}




In [73]:
# ------------------------------------ Grammartical Score -------------------------------------------------------

correct_stmt = "<SOS> While the inaugural address was being delivered from this place, devoted altogether to saving the Union without war <EOS>"
wrong_stmt = "<SOS>  While the inaugural address was being delivered from this place, devoted altogether to saving the Union without war<EOS>"

print("Unigram model")
print(unigramSentCheck(correct_stmt))
print(unigramSentCheck(wrong_stmt))

print("\nBigram model")
print(bigramSentCheck(correct_stmt))
print(bigramSentCheck(wrong_stmt))

print("\nTrigram model")
print(trigramSentCheck(correct_stmt))
print(trigramSentCheck(wrong_stmt))

Unigram model
1.6619203642245514e-08
1.5788243460133238e-08

Bigram model
1.5788243460133238e-08
1.4957283278020963e-08

Trigram model
2.9460084907113255e-06
2.782341352338474e-06


In [74]:
#----------------------------- Predict nth word using ngram ------------------
completeNgram(["is","there"],4)


--------------- completeNgram ----------------

When ['is', 'there'] was given

(('is', 'there', 'any'), 20)
(('is', 'there', 'a'), 9)
(('is', 'there', 'in'), 9)
(('is', 'there', 'anything'), 8)
(('is', 'there', 'not'), 5)
(('is', 'there', '<EOS>'), 4)
(('is', 'there', 'to'), 4)
(('is', 'there', 'such'), 4)
(('is', 'there', 'then'), 3)
(('is', 'there', 'no'), 3)
(('is', 'there', 'may'), 3)
(('is', 'there', 'has'), 3)
(('is', 'there', 'is'), 2)
(('is', 'there', 'it'), 2)
(('is', 'there', 'they'), 2)
(('is', 'there', 'can'), 2)
(('is', 'there', 'so'), 1)
(('is', 'there', 'for'), 1)
(('is', 'there', 'now'), 1)
(('is', 'there', 'but'), 1)
(('is', 'there', 'nobody'), 1)
(('is', 'there', 'something'), 1)
(('is', 'there', 'one'), 1)
(('is', 'there', 'or'), 1)
(('is', 'there', 'room'), 1)
(('is', 'there', 'going'), 1)
(('is', 'there', 'reason'), 1)
(('is', 'there', 'nothing'), 1)
(('is', 'there', 'just'), 1)
(('is', 'there', 'expressed'), 1)
(('is', 'there', 'and'), 1)
(('is', 'there', 'we'),